## Setting Up

### Install dependencies

In [2]:
!pip install -U tensorflow==2.3.0
!pip install -U numpy==1.18.5
!pip install -U Pillow==7.2.0
!pip install -U opencv-python
!pip install -U tf-slim

Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: tensorflow==2.3.0 in /home/lmd/.local/lib/python3.7/site-packages (2.3.0)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


### Imports

In [1]:
import os
import time
import glob
import argparse
import numpy as np
import tensorflow as tf
import cv2

from PIL import Image

os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # Comment out to use GPU for prediction

## Prediction Preparation

### Set all necessary variables

In [7]:
input_folder = "./input/" # Path to folder that contains input images
output_folder = "./output/" # Path to folder to store predicted images
size = "320x320" # Size of image to load into model
threshold = 0.7 # Prediction confidence threshold
model = "./saved_model" # Path to exported tensorflow pb model
label = "./label_map.pbtxt" # Path to label map

### Label map loader

In [8]:
def load_label_map(label_map_path):
    label_map = {}

    with open(label_map_path, "r") as label_file:
        for line in label_file:
            if "id" in line:
                label_index = int(line.split(":")[-1])
                label_name = next(label_file).split(":")[-1].strip().strip("'")
                label_map[label_index] = {"id": label_index, "name": label_name}

    return label_map

### Image to numpy array 

In [9]:
def load_image_into_numpy_array(path, height, width):
    image = Image.open(path).convert("RGB")
    image_shape = np.asarray(image).shape
    image_resized = image.resize((height, width))
    
    return np.array(image_resized), (image_shape[0], image_shape[1])

## Execute Prediction 

### Load label_map, color_map and model 

In [10]:
category_index = load_label_map(label)

color_map = {}
for each_class in range(len(category_index)):
    color_map[each_class] = [int(i) for i in np.random.choice(range(256), size=3)]

start_time = time.time()
trained_model = tf.saved_model.load(model)
print("Model loaded, took {} seconds...".format(time.time() - start_time))

Model loaded, took 25.3473539352417 seconds...


### Loop through images and obtain predictions

In [11]:
## Run prediction on each image
for each_image in glob.glob(os.path.join(input_folder, "*")):
    print("Predicting for {}...".format(each_image))

    height, width = size.split("x")

    ## Returned original_shape is in the format of width, height
    image_resized, origi_shape = load_image_into_numpy_array(
        each_image, int(height), int(width)
    )

    ## The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_resized)

    ## The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    ## Feed image into model
    detections_output = trained_model(input_tensor)

    num_detections = int(detections_output.pop("num_detections"))
    detections = {
        key: value[0, :num_detections].numpy()
        for key, value in detections_output.items()
    }
    detections["num_detections"] = num_detections

    ## Filter out predictions below threshold
    indexes = np.where(detections["detection_scores"] > float(threshold))

    ## Extract predictions
    bboxes = detections["detection_boxes"][indexes]
    classes = detections["detection_classes"][indexes].astype(np.int64)
    scores = detections["detection_scores"][indexes]

    ## Draw Predictions
    image_origi = Image.fromarray(image_resized).resize(
        (origi_shape[1], origi_shape[0])
    )
    image_origi = np.array(image_origi)

    if len(bboxes) != 0:
        for idx, each_bbox in enumerate(bboxes):
            color = color_map.get(classes[idx] - 1)

            ## Draw bounding box
            cv2.rectangle(
                image_origi,
                (
                    int(each_bbox[1] * origi_shape[1]),
                    int(each_bbox[0] * origi_shape[0]),
                ),
                (
                    int(each_bbox[3] * origi_shape[1]),
                    int(each_bbox[2] * origi_shape[0]),
                ),
                color,
                2,
            )

            ## Draw label background
            cv2.rectangle(
                image_origi,
                (
                    int(each_bbox[1] * origi_shape[1]),
                    int(each_bbox[2] * origi_shape[0]),
                ),
                (
                    int(each_bbox[3] * origi_shape[1]),
                    int(each_bbox[2] * origi_shape[0] + 15),
                ),
                color,
                -1,
            )

            ## Insert label class & score
            cv2.putText(
                image_origi,
                "Class: {}, Score: {}".format(
                    str(category_index[classes[idx]]["name"]),
                    str(round(scores[idx], 2)),
                ),
                (
                    int(each_bbox[1] * origi_shape[1]),
                    int(each_bbox[2] * origi_shape[0] + 10),
                ),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.3,
                (0, 0, 0),
                1,
                cv2.LINE_AA,
            )

    ## Save predicted image
    filename = os.path.basename(each_image)
    image_predict = Image.fromarray(image_origi)
    image_predict.save(os.path.join(output_folder, filename))

    print(
        "Saving predicted images to {}...".format(
            os.path.join(output_folder, filename)
        )
    )

Predicting for ./input/362.jpg...
Saving predicted images to ./output/362.jpg...
Predicting for ./input/363.jpg...
Saving predicted images to ./output/363.jpg...
Predicting for ./input/361.jpg...
Saving predicted images to ./output/361.jpg...
